In [ ]:
import sklearn
import nltk

from generateWordFrequency import *
from naiveBayes import *
from fileWriteFunctions import *

In [ ]:
AllClasses = ['story', 'ask_hn', 'show_hn', 'poll']
delta = 0.5
appendClassPrefix = 'prob_'

In [ ]:
trainData = getDataframe(2018)
trainData = addTokenizedColumnofTitle(trainData,None,False)
trainWords = getWordFrequencyDataframe(trainData,AllClasses)

In [ ]:
filteredWordList = getWordListBasedOnPercent(trainWords,Percent = 25)

In [ ]:
trainData = addTokenizedColumnofTitle(trainData,filteredWordList,False)
trainWords = getWordFrequencyDataframe(trainData,AllClasses)

In [ ]:
priorProbabilities = getPriorProbabilities(trainData)
trainWords = obtainDataframeWithClassProbabilities(trainWords, AllClasses, delta, appendClassPrefix)
trainWords = renameModelRows(trainWords, AllClasses, appendClassPrefix)
model = [trainWords, priorProbabilities]

In [ ]:
testData = getDataframe(2019)
testData = addTokenizedColumnofTitle(testData,filteredWordList,False)
testData = testData.reset_index()
testData = generateCondClassProb(testData, model)
testResults = comparePredictions(testData,AllClasses)
writeDataframe(testResults,'wordlength-result.txt')
check = testResults.comparision.value_counts()
accuracy = check[True]/(check[True]+check[False])
print(accuracy)